In [40]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory


from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [41]:
mx.context.num_gpus()

0

In [42]:
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
# df = pd.read_csv('pollution.csv')

In [43]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [44]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [45]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [46]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [47]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [48]:
train_df.shape

(77145, 21)

In [49]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [50]:
train_ds = PandasDataset(
    train_df,
    target = ['TWS_S25A'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

In [51]:
test_ds = PandasDataset(
    test_df,
    target = ['TWS_S25A'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [52]:
K = 48
n_hours = 24*3

In [53]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=64,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
)

model_25a = estimator.train(train_ds)

100%|███████████████████████████████████████████████| 100/100 [10:09<00:00,  6.09s/it, epoch=131/150, avg_epoch_loss=0.0303]


100%|███████████████████████████████████████████████| 100/100 [09:50<00:00,  5.91s/it, epoch=150/150, avg_epoch_loss=0.0286]


#### predicted y

In [54]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [55]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S25A',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model_25a, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.51619506 2.00601888 1.52077997 1.03998339 0.57771868 0.35881054
 0.52220589 1.03249693 1.59031236 2.06607223 2.46756387 2.64769101
 2.47598886 1.96539211 1.4687885  0.97689891 0.45502043 0.13942122
 0.20172973 0.65547103 1.28227496 1.83045673 2.35503912 2.72118115
 2.74220061 2.34345031 1.82783318 1.39300251 0.90731788 0.48292169
 0.36756745 0.66160345 1.21996439 1.70820796 2.16664815 2.5055182
 2.56716442 2.22705388 1.7121731  1.28028953 0.75963956 0.29082555
 0.07755623 0.26537392 0.80664986 1.39799213 1.92397654 2.40955377]
(48,)
------
1
[2.51619506 2.00601888 1.52077997 1.03998339 0.57771868 0.35881054
 0.52220589 1.03249693 1.59031236 2.06607223 2.46756387 2.64769101
 2.47598886 1.96539211 1.4687885  0.97689891 0.45502043 0.13942122
 0.20172973 0.65547103 1.28227496 1.83045673 2.35503912 2.72118115
 2.74220061 2.34345031 1.82783318 1.39300251 0.90731788 0.48292169
 0.36756745 0.66160345 1.21996439 1.70820796 2.16664815 2.5055182
 2.56716442 2.22705388 1.7121731  1.28028953 0

[ 2.51619506  2.00601888  1.52077997  1.03998339  0.57771868  0.35881054
  0.52220589  1.03249693  1.59031236  2.06607223  2.46756387  2.64769101
  2.47598886  1.96539211  1.4687885   0.97689891  0.45502043  0.13942122
  0.20172973  0.65547103  1.28227496  1.83045673  2.35503912  2.72118115
  2.74220061  2.34345031  1.82783318  1.39300251  0.90731788  0.48292169
  0.36756745  0.66160345  1.21996439  1.70820796  2.16664815  2.5055182
  2.56716442  2.22705388  1.7121731   1.28028953  0.75963956  0.29082555
  0.07755623  0.26537392  0.80664986  1.39799213  1.92397654  2.40955377
  2.03294039  1.5355984   1.05136931  0.58255464  0.36213493  0.52856916
  1.0426569   1.6035341   2.07709169  2.47941017  2.66286778  2.49323916
  1.98068392  1.47646093  0.97764111  0.44904408  0.13195169  0.19732338
  0.65406537  1.28467953  1.83557546  2.36439109  2.73644996  2.76088977
  2.36075759  1.84177494  1.4031924   0.91384369  0.48836267  0.3753798
  0.67120141  1.22946775  1.71802819  2.17588735  2.5

[ 2.51619506  2.00601888  1.52077997  1.03998339  0.57771868  0.35881054
  0.52220589  1.03249693  1.59031236  2.06607223  2.46756387  2.64769101
  2.47598886  1.96539211  1.4687885   0.97689891  0.45502043  0.13942122
  0.20172973  0.65547103  1.28227496  1.83045673  2.35503912  2.72118115
  2.74220061  2.34345031  1.82783318  1.39300251  0.90731788  0.48292169
  0.36756745  0.66160345  1.21996439  1.70820796  2.16664815  2.5055182
  2.56716442  2.22705388  1.7121731   1.28028953  0.75963956  0.29082555
  0.07755623  0.26537392  0.80664986  1.39799213  1.92397654  2.40955377
  2.03294039  1.5355984   1.05136931  0.58255464  0.36213493  0.52856916
  1.0426569   1.6035341   2.07709169  2.47941017  2.66286778  2.49323916
  1.98068392  1.47646093  0.97764111  0.44904408  0.13195169  0.19732338
  0.65406537  1.28467953  1.83557546  2.36439109  2.73644996  2.76088977
  2.36075759  1.84177494  1.4031924   0.91384369  0.48836267  0.3753798
  0.67120141  1.22946775  1.71802819  2.17588735  2.5

[ 2.51619506  2.00601888  1.52077997  1.03998339  0.57771868  0.35881054
  0.52220589  1.03249693  1.59031236  2.06607223  2.46756387  2.64769101
  2.47598886  1.96539211  1.4687885   0.97689891  0.45502043  0.13942122
  0.20172973  0.65547103  1.28227496  1.83045673  2.35503912  2.72118115
  2.74220061  2.34345031  1.82783318  1.39300251  0.90731788  0.48292169
  0.36756745  0.66160345  1.21996439  1.70820796  2.16664815  2.5055182
  2.56716442  2.22705388  1.7121731   1.28028953  0.75963956  0.29082555
  0.07755623  0.26537392  0.80664986  1.39799213  1.92397654  2.40955377
  2.03294039  1.5355984   1.05136931  0.58255464  0.36213493  0.52856916
  1.0426569   1.6035341   2.07709169  2.47941017  2.66286778  2.49323916
  1.98068392  1.47646093  0.97764111  0.44904408  0.13195169  0.19732338
  0.65406537  1.28467953  1.83557546  2.36439109  2.73644996  2.76088977
  2.36075759  1.84177494  1.4031924   0.91384369  0.48836267  0.3753798
  0.67120141  1.22946775  1.71802819  2.17588735  2.5

[2.51619506 2.00601888 1.52077997 ... 0.57934529 0.16061537 0.08542192]
(1008,)
------
21
[2.51619506 2.00601888 1.52077997 ... 0.15064621 0.07996678 0.44263721]
(1056,)
------
22
[2.51619506 2.00601888 1.52077997 ... 0.08299247 0.4456071  1.09464788]
(1104,)
------
23
[2.51619506 2.00601888 1.52077997 ... 0.44670227 1.08755016 1.67470503]
(1152,)
------
24
[2.51619506 2.00601888 1.52077997 ... 1.06767476 1.6591202  2.19570565]
(1200,)
------
25
[2.51619506 2.00601888 1.52077997 ... 1.62545836 2.16296887 2.61771202]
(1248,)
------
26
[2.51619506 2.00601888 1.52077997 ... 2.14966464 2.59209752 2.74229431]
(1296,)
------
27
[2.51619506 2.00601888 1.52077997 ... 2.58651972 2.7359066  2.4933908 ]
(1344,)
------
28
[2.51619506 2.00601888 1.52077997 ... 2.71546245 2.46774888 1.93990791]
(1392,)
------
29
[2.51619506 2.00601888 1.52077997 ... 2.46572971 1.93962538 1.4826237 ]
(1440,)
------
30
[2.51619506 2.00601888 1.52077997 ... 1.93193829 1.4624058  0.95116568]
(1488,)
------
31
[2.5161950

...

[2.51619506 2.00601888 1.52077997 ... 2.69620705 2.82368112 2.52284265]
(899232,)
------
18734
[2.51619506 2.00601888 1.52077997 ... 2.85832024 2.55698419 1.92395914]
(899280,)
------
18735
[2.51619506 2.00601888 1.52077997 ... 2.54424167 1.91513407 1.31982315]
(899328,)
------
18736
[2.51619506 2.00601888 1.52077997 ... 1.8340807  1.24190509 0.62098938]
(899376,)
------
18737
[2.51619506 2.00601888 1.52077997 ... 1.22860634 0.59859687 0.01624095]
(899424,)
------
18738
[ 2.51619506  2.00601888  1.52077997 ...  0.59995526  0.01650784
 -0.25172845]
(899472,)
------
18739
[ 2.51619506  2.00601888  1.52077997 ...  0.02004614 -0.24597184
 -0.08516208]
(899520,)
------
18740
[ 2.51619506  2.00601888  1.52077997 ... -0.20714021 -0.05057098
  0.56777698]
(899568,)
------
18741
[ 2.51619506  2.00601888  1.52077997 ... -0.06997707  0.54298377
  1.25711763]
(899616,)
------
18742
[2.51619506 2.00601888 1.52077997 ... 0.49167156 1.21237183 1.85119927]
(899664,)
------
18743
[2.51619506 2.00601888

[2.51619506 2.00601888 1.52077997 ... 1.9904896  2.49950385 2.73274255]
(903360,)
------
18820
[2.51619506 2.00601888 1.52077997 ... 2.49230242 2.7302618  2.55728841]
(903408,)
------
18821
[2.51619506 2.00601888 1.52077997 ... 2.72204137 2.55371952 1.95325041]
(903456,)
------
18822
[2.51619506 2.00601888 1.52077997 ... 2.53411269 1.93299925 1.35825253]
(903504,)
------
18823
[2.51619506 2.00601888 1.52077997 ... 1.91799963 1.33996582 0.76413679]
(903552,)
------
18824
[2.51619506 2.00601888 1.52077997 ... 1.32627654 0.75156528 0.21505423]
(903600,)
------
18825
[ 2.51619506  2.00601888  1.52077997 ...  0.73394412  0.20080332
 -0.02720734]
(903648,)
------
18826
[ 2.51619506  2.00601888  1.52077997 ...  0.21346003 -0.0151967
  0.16595583]
(903696,)
------
18827
[ 2.51619506e+00  2.00601888e+00  1.52077997e+00 ... -3.03804874e-04
  1.74796283e-01  7.27389514e-01]
(903744,)
------
18828
[2.51619506 2.00601888 1.52077997 ... 0.18748342 0.73479778 1.3350029 ]
(903792,)
------
18829
[2.516

[2.51619506 2.00601888 1.52077997 ... 1.99260759 2.087116   1.90217268]
(907584,)
------
18908
[2.51619506 2.00601888 1.52077997 ... 2.08727932 1.89517021 1.47075522]
(907632,)
------
18909
[2.51619506 2.00601888 1.52077997 ... 1.893929   1.47562039 1.03103757]
(907680,)
------
18910
[2.51619506 2.00601888 1.52077997 ... 1.50635386 1.06080663 0.59074569]
(907728,)
------
18911
[2.51619506 2.00601888 1.52077997 ... 1.06862247 0.60363299 0.23963362]
(907776,)
------
18912
[2.51619506 2.00601888 1.52077997 ... 0.59700775 0.248564   0.10987151]
(907824,)
------
18913
[2.51619506 2.00601888 1.52077997 ... 0.22280824 0.08890835 0.25184751]
(907872,)
------
18914
[2.51619506 2.00601888 1.52077997 ... 0.07541368 0.23747504 0.65641719]
(907920,)
------
18915
[2.51619506 2.00601888 1.52077997 ... 0.24999063 0.66268384 1.13064253]
(907968,)
------
18916
[2.51619506 2.00601888 1.52077997 ... 0.65890867 1.13135636 1.51887095]
(908016,)
------
18917
[2.51619506 2.00601888 1.52077997 ... 1.12843037 1

[2.51619506 2.00601888 1.52077997 ... 2.13448644 2.00368953 1.65471137]
(911808,)
------
18996
[2.51619506 2.00601888 1.52077997 ... 2.01409626 1.66760743 1.30108607]
(911856,)
------
18997
[2.51619506 2.00601888 1.52077997 ... 1.62242901 1.25372314 0.85243851]
(911904,)
------
18998
[2.51619506 2.00601888 1.52077997 ... 1.28511667 0.91410476 0.57506227]
(911952,)
------
18999
[2.51619506 2.00601888 1.52077997 ... 0.96614391 0.63318449 0.44841042]
(912000,)
------
19000
[2.51619506 2.00601888 1.52077997 ... 0.6217832  0.44008732 0.4262042 ]
(912048,)
------
19001
[2.51619506 2.00601888 1.52077997 ... 0.40112206 0.40226355 0.70141429]
(912096,)
------
19002
[2.51619506 2.00601888 1.52077997 ... 0.36457539 0.68154764 1.17459977]
(912144,)
------
19003
[2.51619506 2.00601888 1.52077997 ... 0.62010354 1.12562501 1.54312289]
(912192,)
------
19004
[2.51619506 2.00601888 1.52077997 ... 1.11771202 1.54049683 1.89421332]
(912240,)
------
19005
[2.51619506 2.00601888 1.52077997 ... 1.55048883 1

[2.51619506 2.00601888 1.52077997 ... 1.94920933 1.49838698 1.0769645 ]
(916032,)
------
19084
[2.51619506 2.00601888 1.52077997 ... 1.47716582 1.06500101 0.64422339]
(916080,)
------
19085
[2.51619506 2.00601888 1.52077997 ... 1.03392208 0.6194272  0.25190565]
(916128,)
------
19086
[2.51619506 2.00601888 1.52077997 ... 0.60157824 0.23303586 0.08288101]
(916176,)
------
19087
[2.51619506 2.00601888 1.52077997 ... 0.18462782 0.03158766 0.18994816]
(916224,)
------
19088
[2.51619506 2.00601888 1.52077997 ... 0.02581678 0.17330702 0.6136179 ]
(916272,)
------
19089
[2.51619506 2.00601888 1.52077997 ... 0.17722712 0.61514783 1.06529295]
(916320,)
------
19090
[2.51619506 2.00601888 1.52077997 ... 0.5718413  1.03446007 1.43875599]
(916368,)
------
19091
[2.51619506 2.00601888 1.52077997 ... 1.01857543 1.43129778 1.78141499]
(916416,)
------
19092
[2.51619506 2.00601888 1.52077997 ... 1.41820896 1.7568084  1.91105843]
(916464,)
------
19093
[2.51619506 2.00601888 1.52077997 ... 1.75742614 1

In [56]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19168, 48)

In [57]:
test_ds_predict_q50_supervised[0:3]

array([[2.51619506, 2.00601888, 1.52077997, 1.03998339, 0.57771868,
        0.35881054, 0.52220589, 1.03249693, 1.59031236, 2.06607223,
        2.46756387, 2.64769101, 2.47598886, 1.96539211, 1.4687885 ,
        0.97689891, 0.45502043, 0.13942122, 0.20172973, 0.65547103,
        1.28227496, 1.83045673, 2.35503912, 2.72118115, 2.74220061,
        2.34345031, 1.82783318, 1.39300251, 0.90731788, 0.48292169,
        0.36756745, 0.66160345, 1.21996439, 1.70820796, 2.16664815,
        2.5055182 , 2.56716442, 2.22705388, 1.7121731 , 1.28028953,
        0.75963956, 0.29082555, 0.07755623, 0.26537392, 0.80664986,
        1.39799213, 1.92397654, 2.40955377],
       [2.03294039, 1.5355984 , 1.05136931, 0.58255464, 0.36213493,
        0.52856916, 1.0426569 , 1.6035341 , 2.07709169, 2.47941017,
        2.66286778, 2.49323916, 1.98068392, 1.47646093, 0.97764111,
        0.44904408, 0.13195169, 0.19732338, 0.65406537, 1.28467953,
        1.83557546, 2.36439109, 2.73644996, 2.76088977, 2.36075759,
   

#### actual y

In [58]:
from helper import series_to_supervised, stage_series_to_supervised

In [59]:
test_df_supervised = series_to_supervised(test_df[['WS_S1']], n_hours, K)

In [60]:
test_df_supervised.iloc[:, -K:].shape

(19168, 48)

In [61]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [62]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+38),var1(t+39),var1(t+40),var1(t+41),var1(t+42),var1(t+43),var1(t+44),var1(t+45),var1(t+46),var1(t+47)
77217,2.53,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,...,1.62,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48
77218,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,...,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80
77219,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,...,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70
77220,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,...,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19
77221,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,2.05,...,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,1.95,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,...,2.36,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24
96381,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,...,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62
96382,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,...,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98
96383,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,1.99,...,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98,2.13


#### performance on test set

In [63]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.149
RMSE = 0.2
